<a href="https://colab.research.google.com/github/anarossati/SQL_Python_Ocean_06_2021/blob/main/python_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Banco de Dados



* Componentes
  * Base de dados
  * Sistema Gerenciador de Bando de Dados (SGDB)
  * Linguagem de Manipulação
    *  Structered Query Language (SQL)
  * Programas adicionais

* Arquitetura
  * Cliente-Servidor

## Modelo Relacional
  
| nome | tipo | level | capturado em|
|---- |----|-----|----|
|Pikachu| Elétrico| 5| 08 jun 2021|
|Bulbassauro| Grama| 10| 10 fev 1990|
|Squirtle| Água| 12| 23 out 2015|

Tabela ou Relação

Linhas, Intâncias, Entradas, Registros, Tupla

Colunas, atributos, características

## Implementação em SQL

* dialetos: MySQL, PostreSQL, MariaDB, Oracle etc

* MySQL online: https://paiza.io/en/projects/new?language=mysql
* PostgreSQL as a Service: https://www.elephantsql.com/
* Sqlite e PostgreSQL: https://sqliteonline.com/

#### Criando um banco de dados dentro do SGDB
* MySQL
      CREATE DATABASE [IF NOT EXISTS] db_name
*PostgreSQL
      CREATE DATABASE name
*SQLite
      # Não tem (nesse formato)

### Linguagem de Descrição de Dados (DDL)


#### CREATE (Criar)
      CREATE [TEMPORARY] TABLE [IF NOT EXISTS] tbl_name
      [column_name column_type column_constraint, ...]
      [table_constraints, ...]

[MySQL types](https://dev.mysql.com/doc/refman/8.0/en/data-types.html) 
\- [PostrgreSQL types](https://www.postgresql.org/docs/9.5/datatype.html)
\- [SQLite types](https://www.sqlite.org/datatype3.html)

* Most used Constraints:

#### ALTER (alterar)
      ALTER TABLE tbl_name [alter_option, ...]

#### DROP (descartar)
      DROP TABLE [ IF EXISTS ] tbl_name [, ...]


### Linguagem de Manipulação de Dados (DML)


#### INSERT (inserir)

      INSERT  INTO tbl_name[(col_name [, col_name] ...)]
      VALUES (value_list) [, (value_list)] ...```


#### UPDATE (atualizar)
      UPDATE table_reference
      SET assignment_list
      [WHERE where_condition]

#### DELETE (deletar) 
      DELETE FROM tbl_name [WHERE where_condition]

### Linguagem de Consulta de Dados (DQL)


#### SELECT (SELECIONAR)

      SELECT [ALL | ISTINCT | DISTINCTROW ]
      select_expr [, select_expr] ...
      [FROM table_references
      [WHERE where_condition]
      [GROUP BY {col_name | expr | position}, ...]
      [HAVING where_condition]
      [ORDER BY {col_name | expr | position} [ASC | DESC], ...]
      [LIMIT {[offset,] row_count | row_count OFFSET offset}]

#### JOINS

### Linguagem de Transação de Dados (DTL)
### Linguagem de Controle de Dados (DCL)

# Integração com Python


In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("banco_teste.db")

In [3]:
con.execute("""CREATE TABLE Pokemons(
  id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
  nome VARCHAR(20)
  )""")

In [4]:
con.execute("INSERT INTO Pokemons (nome) VALUES ('Pikachu')")

In [5]:
rs = con.execute("SELECT * FROM Pokemons")

In [6]:
list(rs)

[(1, 'Pikachu')]

## SQLAlchemy

É um *framework* desenvolvido para permitir programar aplicações que usem Banco de dados, de maneira inpedepende e com diferentes tipos de abstração:

* Raw SQL (SQL Bruto)
* SQL Expression Language (Linguagem de Expressão SQL)
* Object-Relational Mapping - ORM (Mapeamento Objeto-Relacional)

### Raw SQL

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

In [8]:
# Cria a engine para o banco de dados
eng = create_engine("sqlite:///cars.db")
#eng_postgres = create_engine("postgresql://") #url d banco criado na nuvem

In [9]:
# abre uma conexão com o banco e executa código SQL
with eng.connect() as con:
  con.execute(text('DROP TABLE IF EXISTS Cars'))
  con.execute(text('''CREATE TABLE Cars(Id INTEGER PRIMARY KEY, 
                   Name TEXT, Price INTEGER)'''))

In [10]:
with eng.connect() as con:
  con.execute(text("INSERT INTO Cars(Id, Name, Price) VALUES (9, 'Fusca', 1000)"))

In [11]:
data = [{ "Id": 1, "Name": "Audi", "Price": 52642 },
        { "Id": 2, "Name": "Mercedes", "Price": 57127 },
        { "Id": 3, "Name": "Skoda", "Price": 9000 },
        { "Id": 4, "Name": "Volvo", "Price": 29000 },
        { "Id": 5, "Name": "Bentley", "Price": 350000 },
        { "Id": 6, "Name": "Citroen", "Price": 21000 },
        { "Id": 7, "Name": "Hummer", "Price": 41400 },
        { "Id": 8, "Name": "Volkswagen", "Price": 21600 }
]
# Inserindo dados
with eng.connect() as con:
  for line in data:
    con.execute(text("""INSERT INTO Cars(Id, Name, Price) VALUES(:Id, :Name, :Price)"""), **line)

In [12]:
# Recebendo dados
with eng.connect() as con:
  rs = con.execute(text("SELECT * FROM Cars"))
  print(rs.fetchone())
  print(rs.fetchmany(3))
  print(rs.fetchall())

(1, 'Audi', 52642)
[(2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000)]
[(5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (9, 'Fusca', 1000)]


### SQL Expression Language

#### Definição de Schemas

In [13]:
from sqlalchemy import Table, Column, Integer, String, MetaData

In [14]:
# Cria um ojeto de metadata. Metadatas são coleções de tabelas e sua infos
meta = MetaData()

In [15]:
#Cria um objeto do tipo tabela que abstrai uma tabela no Banco de dados
cars = Table('Cars', meta,
     Column('Id', Integer, primary_key=True),
     Column('Name', String),
     Column('Price', Integer)
)

In [16]:
print ("The Name column:")
print (cars.columns.Name)
print (cars.c.Name)

print ("Columns: ")
for col in cars.c:
    print (col)
    
print ("Primary keys:")
for pk in cars.primary_key:
    print (pk)    

print ("The Id column:")
print (cars.c.Id.name)
print (cars.c.Id.type)
print (cars.c.Id.nullable)
print (cars.c.Id.primary_key)

The Name column:
Cars.Name
Cars.Name
Columns: 
Cars.Id
Cars.Name
Cars.Price
Primary keys:
Cars.Id
The Id column:
Id
INTEGER
False
True


In [17]:
meta.reflect(bind=eng)
for table in meta.tables:
    print(table)

Cars


In [18]:
from sqlalchemy import inspect

In [19]:
insp = inspect(eng)
print(insp.get_table_names())
print(insp.get_columns("Cars"))
print(insp.get_pk_constraint("Cars")) 

['Cars']
[{'name': 'Id', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'Name', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'Price', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]
{'constrained_columns': ['Id'], 'name': None}


#### Expression Language

In [20]:
from sqlalchemy.sql import select 

In [21]:
# SELECT
with eng.connect() as con:
  stm = select([cars])
  rs = con.execute(stm) 
  print(rs.fetchall())

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (9, 'Fusca', 1000)]


In [22]:
print(stm)

SELECT "Cars"."Id", "Cars"."Name", "Cars"."Price" 
FROM "Cars"


In [23]:
# meta2 = MetaData(eng_postgres)
# meta2.reflect(bind=eng_postgres)
# cars_postgres = Table('cars', meta2, autoload=True)  

In [24]:
# SELECT
# with eng.connect() as con:
#   stm = select([cars_postgres])
#   rs = con.execute(stm) 
#   print(rs.fetchall())

In [25]:
# LIMIT
with eng.connect() as con:
  stm = select([cars.c.Name, cars.c.Price]).limit(3)
  rs = con.execute(stm) 
  print(rs.fetchall())

[('Audi', 52642), ('Mercedes', 57127), ('Skoda', 9000)]


In [26]:
print(stm)

SELECT "Cars"."Name", "Cars"."Price" 
FROM "Cars"
 LIMIT :param_1


In [27]:
from sqlalchemy.sql import and_, or_  

In [28]:
# WHERE
with eng.connect() as con:
  stm = select([cars]).where(and_(cars.c.Price > 10000, cars.c.Price < 40000))
  rs = con.execute(stm) 
  print(rs.fetchall())

[(4, 'Volvo', 29000), (6, 'Citroen', 21000), (8, 'Volkswagen', 21600)]


In [29]:
#LIKE
with eng.connect() as con:
  stm = select([cars]).where(cars.c.Name.like('%en'))
  rs = con.execute(stm) 
  print(rs.fetchall())

[(6, 'Citroen', 21000), (8, 'Volkswagen', 21600)]


In [30]:
from sqlalchemy.sql import asc, desc

In [31]:
# Order By
with eng.connect() as con:  
  s = select([cars]).order_by(asc(cars.c.Name))
  rs = con.execute(s) 
  for row in rs:
    print(row['Id'], row['Name'], row['Price'])

1 Audi 52642
5 Bentley 350000
6 Citroen 21000
9 Fusca 1000
7 Hummer 41400
2 Mercedes 57127
3 Skoda 9000
8 Volkswagen 21600
4 Volvo 29000


In [32]:

# ler dos arquivos
with eng.connect() as con:
  authors = Table('Authors', meta, autoload=True)
  books = Table('Books', meta, autoload=True)
  stm = select([authors.join(books)])
  rs = con.execute(stm) 
  for row in rs:
    print(row['Name'], row['Title'])

UnboundExecutionError: ignored

### Object-Relational Mapping (ORM)

In [58]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

In [59]:
# Classes Bases e Sessão
Base = declarative_base(bind=eng)
Session = sessionmaker(bind=eng)

In [ ]:
class Car(Base):
  __tablename__ = "Cars"

  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String, unique=True)  
  price = Column(Integer, nullable=False)

  def __repr__(self):
    return f"<Car:{self.name}>"

In [ ]:
Base.metadata.create_all()        
ses = Session()  

In [ ]:
try:
  ses.add(Car(name='Audi', price=52642))
  ses.commit()
except IntegrityError:
  print("Já existe um carro com esse nome e valor")
  ses.rollback()
ses.close()

In [ ]:
with Session() as s:
  s.add_all([ 
    Car(id=2, name='Mercedes', price=57127),
    Car(id=3, name='Skoda', price=9000),
    Car(id=4, name='Volvo', price=29000),
    Car(id=5, name='Bentley', price=350000),
    Car(id=6, name='Citroen', price=21000),
    Car(id=7, name='Hummer', price=41400),
    Car(id=8, name='Volkswagen', price=21600)
  ])
  s.commit()

In [ ]:
# Fazendo a query
with Session() as s:
  rs = s.query(Car).all()
  for car in rs:
    print(car.name, car.price)

In [ ]:
with Session() as s:
  rs = s.query(Car).filter(Car.name.like('%en'))
  for car in rs:
    print(car.name, car.price)

In [ ]:
with Session() as s:
  rs = s.query(Car).filter(Car.price >10000).limit(2)
  for car in rs:
    print(car.name, car.price)

In [52]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [60]:
class Author(Base):
  __tablename__ = "Authors"
 
  author_id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String)  
  books = relationship("Book")

  def __init__(self, name):
    self.name = name

  def __repr__(self):
    return f"<Author: {self.name}>"

class Book(Base):
  __tablename__ = "Books"
 
  book_id = Column(Integer, primary_key=True, autoincrement=True)
  title = Column(String)      
  author_id = Column(Integer, ForeignKey("Authors.author_id"))    
  author = relationship("Author")

  def __init__(self, title, author_id):
    self.title = title
    self.author_id = author_id 

  def __repr__(self):
    return f"<Book: {self.title} - {self.author.name}>"                 

In [61]:
Base.metadata.create_all()

In [62]:
with Session() as s:
  s.add_all([ 
    Author('Jane Austen'),
    Author('Leo Tolstoy'),
    Author('Joseph Heller'),
    Author('Charles Dickens'),
    Book('Emma',1),
    Book('War and Peace',2),
    Book('Catch XII',3),
    Book('David Copperfield',4),
    Book('Good as Gold',3),
    Book('Anna Karenia',2)
  ])
  s.commit()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SAWarning: relationship 'Book.author' will copy column Authors.author_id to column Books.author_id, which conflicts with relationship(s): 'Author.books' (copies Authors.author_id to Books.author_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
with Session() as s:
  res = s.query(Author).all()
  print(res)

[<Author: Jane Austen>, <Author: Leo Tolstoy>, <Author: Joseph Heller>, <Author: Charles Dickens>]


In [64]:
with Session() as s:
  res = s.query(Book).all()
  print(res)

[<Book: Emma - Jane Austen>, <Book: War and Peace - Leo Tolstoy>, <Book: Catch XII - Joseph Heller>, <Book: David Copperfield - Charles Dickens>, <Book: Good as Gold - Joseph Heller>, <Book: Anna Karenia - Leo Tolstoy>]


In [65]:
with Session() as s:
  res = s.query(Author).filter(Author.name=="Leo Tolstoy").first()
  print(res)
  for book in res.books:
    print (book.title)

<Author: Leo Tolstoy>
War and Peace
Anna Karenia


In [66]:
with Session() as s:
  res = s.query(Book).filter(Book.title=="Emma").first()    
  print(res.author.name)

Jane Austen
